In [1]:
import pandas as pd
import csv
import requests
from webcrawl import WebCrawl

In [2]:
movie = pd.read_csv('movie_without_boxoffice.csv')

In [3]:
len(movie[(movie.YEAR >= 1990) & (movie.YEAR <= 2015) & (movie.COUNTRY.str.contains('USA') \
                                                       & (movie.LANGUAGE.str.contains('English')))])

10151

In [4]:
filteriengCSV = movie[(movie.YEAR >= 1990) & (movie.YEAR <= 2015) & (movie.COUNTRY.str.contains('USA') \
                                                       & (movie.LANGUAGE.str.contains('English')))]

In [5]:
requiredDetails = pd.DataFrame(filteriengCSV)

In [6]:
# requiredDetails.to_csv('final_with_boxoffice.csv',columns=['TMDB ID', 'IMDB ID', 'TITLE', 'YEAR', 'GENRE', 'RATING',\
#                                            'RELEASED','ACTORS','AWARDS','COUNTRY','LANGUAGE'],index=False)

In [7]:
requiredDetails.head()

,TMDB ID,IMDB ID,TITLE,YEAR,GENRE,RATING,RELEASED,ACTORS,AWARDS,COUNTRY,LANGUAGE
0,862,tt0114709,Toy Story,1995,"Animation, Adventure, Comedy",8.3,22 Nov 1995,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",Nominated for 3 Oscars. Another 23 wins & 18 n...,USA,English
1,8844,tt0113497,Jumanji,1995,"Action, Adventure, Family",6.9,15 Dec 1995,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",4 wins & 9 nominations.,USA,"English, French"
2,15602,tt0113228,Grumpier Old Men,1995,"Comedy, Romance",6.6,22 Dec 1995,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",2 wins & 2 nominations.,USA,English
3,31357,tt0114885,Waiting to Exhale,1995,"Comedy, Drama, Romance",5.7,22 Dec 1995,"Whitney Houston, Angela Bassett, Loretta Devin...",8 wins & 8 nominations.,USA,English
4,11862,tt0113041,Father of the Bride Part II,1995,"Comedy, Family, Romance",5.9,08 Dec 1995,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Nominated for 1 Golden Globe. Another 1 win & ...,USA,English


In [8]:
# for i in requiredDetails.values[0:10]:
#     print (type(i[0]),type(i[1]),i[3])

In [15]:
listWithBoxOffice = [[],[],[]]

In [16]:
def extract_boxOffice(imdbid,year):
    url = 'http://www.omdbapi.com/?i=' + imdbid
    boxOffice = 'N/A' 
    try:
        jsonResponse = requests.get(url)
        if 'N/A' == jsonResponse.json()['BoxOffice']:
            boxOffice = WebCrawl.extractBoxOfficeByIMDB(imdbid)
        else:
            box = ''
            for i in jsonResponse.json()['BoxOffice'].split('$')[1].split(','):
                box += i
                boxOffice = box
            
        # print (boxOffice)
    except requests.exceptions.RequestException as e:
        print ('Exception: {}'.format(e))
        # print (imdbid)
    except KeyError:
        boxOffice = WebCrawl.extractBoxOfficeByIMDB(imdbid)
        # print (imdbid)
        
    listWithBoxOffice[0].append(imdbid)
    listWithBoxOffice[1].append(boxOffice)
    listWithBoxOffice[2].append(year)

In [17]:
for i in requiredDetails.values[0:1000]:
#     passing imdbid, year
    extract_boxOffice(i[1], i[3])

In [18]:
len(listWithBoxOffice[0])

1000

In [19]:
# listWithBoxOffice

In [20]:
with open('movie_with_boxoffice_1000.csv', 'w') as csvfile:
    fieldnames = ['IMDB ID','BOX OFFICE','YEAR']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for i in range(len(listWithBoxOffice[0])):
        writer.writerow({'IMDB ID': listWithBoxOffice[0][i], 'BOX OFFICE': listWithBoxOffice[1][i], \
                        'YEAR': listWithBoxOffice[2][i]})

In [7]:
r = requests.get('http://www.omdbapi.com/?i=tt0114319')

In [24]:
if 'N/A' == r.json()['BoxOffice']:
    print ('Yes')

Yes


In [26]:
x = requests.get('http://www.omdbapi.com/?i=tt2771200')

In [37]:
box = ''
for i in x.json()['BoxOffice'].split('$')[1].strip('.00').split(','):
    box += i
    

In [71]:
movie = pd.read_csv('movie.csv')

In [72]:
movie.YEAR.isnull().value_counts(),movie.RELEASED.isnull().value_counts()

(False    82
 Name: YEAR, dtype: int64, False    82
 Name: RELEASED, dtype: int64)

In [74]:
movie.BOXOFFICE.isnull().value_counts(), movie.GENRE.isnull().value_counts()

(False    82
 Name: BOXOFFICE, dtype: int64, False    82
 Name: GENRE, dtype: int64)

In [75]:
movie.RATING.isnull().value_counts()

False    82
Name: RATING, dtype: int64